In [7]:
import pandas as pd
from scipy import sparse
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

In [8]:
y_train = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\y_train.csv')

In [9]:
y_train.columns

Index(['Diabetes'], dtype='object')

In [10]:
y_test = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\y_test.csv')

In [84]:
x_train = pd.read_csv(f'C:\Projects_ciência_dados\DIABETES\data\processed\x_train.csv')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 58-59: truncated \xXX escape (2159860071.py, line 1)

In [11]:
x_train = sparse.load_npz(r'C:\Projects_ciência_dados\DIABETES\data\processed\x_train.npz')

In [12]:
x_test = sparse.load_npz(r'C:\Projects_ciência_dados\DIABETES\data\processed\x_test.npz')

#### Grid trabalha com dados completos, não precisa separar em treino e teste, pois ele já faz isso com a validação cruzada

In [13]:
x_trainDF = pd.DataFrame(x_train.toarray())

In [14]:
x_testDF = pd.DataFrame(x_test.toarray())

In [15]:
x_geral = pd.concat([x_trainDF,x_testDF], axis=0)

In [16]:
y_geral = pd.concat([y_train,y_test], axis=0)

#### Gaussian

In [17]:
from sklearn.naive_bayes import BernoulliNB

In [18]:
bernoulli = BernoulliNB()
bernoulli.fit(x_train, y_train)
previsao_bernoulli = bernoulli.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
acuracia_bernoulli = accuracy_score(previsao_bernoulli, y_test)
print(f"Acurácia bernoulli: {acuracia_bernoulli}")

Acurácia bernoulli: 0.82


In [20]:
print(classification_report(previsao_bernoulli,y_test))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84      1338
           1       0.83      0.77      0.80      1162

    accuracy                           0.82      2500
   macro avg       0.82      0.82      0.82      2500
weighted avg       0.82      0.82      0.82      2500



#### Árvores

In [21]:
parametros = {
    "max_depth":[500, 600, 650],
    "min_samples_split":[2,5,10],
    "min_samples_leaf":[1,5,10],
    "criterion":["gini", "entropy"]
}

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#### Grid trabalha com dataset completo

In [25]:
randomized = RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42), param_distributions=parametros, verbose=True, n_jobs=4, cv=3, n_iter=20, random_state=42)
randomized.fit(x_train,y_train)
melhores_parametros = randomized.best_params_
melhor_resultado = randomized.best_score_


Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [26]:
print(melhores_parametros)

{'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 650, 'criterion': 'gini'}


In [27]:
print(melhor_resultado)

0.8811999999999999


In [ ]:
grid = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=parametros)
grid.fit(x_geral, y_geral)
melhores_parametros = grid.best_params_
melhor_resultado = grid.best_score_

In [ ]:
print(melhores_parametros)

{'criterion': 'log_loss', 'max_depth': 500, 'min_samples_leaf': 1, 'min_samples_split': 10}


In [ ]:
print(melhor_resultado)

0.8238666666666667


In [31]:
arbol1 = DecisionTreeClassifier()
arbol1.fit(x_train, y_train)
previsao_arbol1 = arbol1.predict(x_test)

In [33]:
cross_arbol1 = cross_val_score(arbol1,x_train,y_train, cv=5)
print(cross_arbol1.mean())

0.9069333333333333


In [28]:
arbol = DecisionTreeClassifier(max_depth=15, min_samples_split=50, min_samples_leaf=20, max_features='sqrt', random_state=42)
arbol.fit(x_train,y_train)
previsao_arbol = arbol.predict(x_test)

In [29]:
print(accuracy_score(previsao_arbol,y_test))

0.5648


In [30]:
print(classification_report(previsao_arbol,y_test))

              precision    recall  f1-score   support

           0       1.00      0.56      0.72      2500
           1       0.00      0.00      0.00         0

    accuracy                           0.56      2500
   macro avg       0.50      0.28      0.36      2500
weighted avg       1.00      0.56      0.72      2500



c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


#### Profundidade da árvore

In [ ]:
arbol.get_depth()

557

#### Numero de folhas

In [ ]:
arbol.get_n_leaves()

np.int64(596)

In [ ]:
arbol.tree_

##### Serve para ver as features mais importantes, mas ele perdeu o nome das features

In [ ]:
importances = pd.DataFrame({
    "features":x_train_DF.columns,
    "importancia":arbol.feature_importances_
}).sort_values(by="importancia", ascending=False)

In [ ]:
importances.head()

,features,importancia
1869,1869,0.231530
1873,1873,0.004645
1876,1876,0.004037
1877,1877,0.003153
1868,1868,0.002357


#### Regressão logística

In [34]:
from sklearn.linear_model import LogisticRegression

In [35]:
logistic = LogisticRegression(max_iter=1000)
logistic.fit(x_train, y_train)
previsao_logistic = logistic.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
print(accuracy_score(previsao_logistic,y_test))

0.8944


In [37]:
print(classification_report(previsao_logistic,y_test))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1380
           1       0.89      0.87      0.88      1120

    accuracy                           0.89      2500
   macro avg       0.89      0.89      0.89      2500
weighted avg       0.89      0.89      0.89      2500



In [38]:
cross_logistic = cross_val_score(logistic, x_train, y_train, cv=5)
cross_logistic.mean()

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d

np.float64(0.8713333333333333)

#### Random

In [40]:
from sklearn.ensemble import RandomForestClassifier

#### Verificar se o random forest está overfitando ou underfitando, verificar os parâmetros default

In [58]:
from scipy.stats import randint

In [80]:
parametros = {
    "n_estimators": randint(20,100),
    "min_samples_split": randint(10,30),
    "min_samples_leaf": randint(10,30),
    "max_features": ['log2','sqrt'],
    "max_depth": randint(10,20)
}

In [81]:
randomized_random = RandomizedSearchCV(estimator=RandomForestClassifier(), param_distributions=parametros, n_jobs=4, verbose=True)
randomized_random.fit(x_train,y_train)
melhores_parametros = randomized_random.best_params_
melhor_resultado = randomized_random.best_score_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [82]:
print(melhor_resultado)

0.5573333333333333


In [83]:
print(melhores_parametros)

{'max_depth': 11, 'max_features': 'log2', 'min_samples_leaf': 25, 'min_samples_split': 24, 'n_estimators': 27}


In [77]:
random = RandomForestClassifier(verbose=True, n_jobs=4)
random.fit(x_train,y_train)
previsao_random = random.predict(x_test)
previsao_ingenua_random = random.predict(x_train)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [78]:
print(accuracy_score(previsao_random,y_test))

0.7264


In [79]:
print(accuracy_score(previsao_ingenua_random,y_train))

1.0


In [47]:
cross_random = cross_val_score(random, x_train,y_train,cv=5)
print(f"Cross random: {cross_random.mean()}")

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(

Cross random: 0.7118666666666666


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [56]:
print(random.max_depth)

None


#### Número de árvores

In [52]:
print(random.n_estimators)

100


#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(x_train,y_train)
previsao_KNN = KNN.predict(x_test)

In [ ]:
print(accuracy_score(previsao_KNN,y_test))

0.8292


In [ ]:
x_train_array = x_train.toarray()

In [ ]:
x_train_array

array([[0.        , 2.000001  , 2.30847842, ..., 4.32174931, 2.00019603,
        2.00108989],
       [0.        , 2.000001  , 2.30847842, ..., 5.16042265, 2.00019603,
        2.00108989],
       [0.        , 2.000001  , 0.        , ..., 2.7853444 , 0.        ,
        0.        ],
       ...,
       [0.        , 2.000001  , 2.30847842, ..., 4.5757236 , 0.        ,
        2.00108989],
       [2.000001  , 0.        , 2.30847842, ..., 5.48137917, 2.00019603,
        2.00108989],
       [2.000001  , 0.        , 2.30847842, ..., 3.72449109, 2.00019603,
        0.        ]], shape=(7500, 1880))

##### SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
parametros = {
    "C":[1,2,4],
    "kernel":['poly','rbf','sigmoid']
}

In [ ]:
grid = GridSearchCV(estimator=SVC(),param_grid=parametros)
grid.fit(x_geral,y_geral)
melhores_parametros = grid.best_estimator_
melhor_resultado = grid.best_score_

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d

In [ ]:
print(melhor_resultado)

0.8939


In [ ]:
print(melhores_parametros)

SVC(C=4)


In [ ]:
SVM = SVC()
SVM.fit(x_train,y_train)
previsao_SVM = SVM.predict(x_test)
previsao_ingenua = SVM.predict(x_train)

In [ ]:
print(accuracy_score(previsao_ingenua,y_train))

0.9349333333333333


In [ ]:
cross_SVM = cross_val_score(SVM,x_train,y_train,cv=5)

In [ ]:
print(cross_SVM.mean())

0.8386666666666667


In [ ]:
print(accuracy_score(previsao_SVM,y_test))

0.8564


##### xgboost

In [ ]:
#### Ocorreu underfitting

In [ ]:
from xgboost import XGBClassifier

In [ ]:
XG = XGBClassifier()
XG.fit(x_train,y_train)
cross_xg = cross_val_score(XG,x_train,y_train,cv=5)
previsao_XG = XG.predict(x_test)
previsao_ingenua = XG.predict(x_train)

In [ ]:
print(accuracy_score(previsao_ingenua,y_train)) 

0.8432


In [ ]:
print(cross_xg.mean())

0.7518666666666667


In [ ]:
print(accuracy_score(previsao_XG,y_test))

0.7496
